# Gaia Data I/O and Processing Examples

This document contains examples of Gaia's file I/O capabilities and geospatial processes.

The original Python notebook is available in the Gaia source code under the docs/examples folder.

In [40]:
# Import required modules
from gaia.geo.geo_inputs import *
from gaia.geo.processes_vector import *
from gaia.geo.processes_raster import *
from gaia import formats
from sqlalchemy import Table, Column, Integer, String, MetaData, create_engine, select, text
from geoalchemy2 import Geometry
import json
import folium #for mapping
from folium.plugins import ImageOverlay

## Data

- Create VectorFileIO objects from files containing GeoJSON.
- Create a FeatureIO object containing one point.
- Create RasterFileIO objects from GeoTiff image files.

In [2]:
# Roads in Iraq
iraq_allroads = VectorFileIO(uri='../tests/data/iraq_roads.geojson')

# Roads in Iraq where type = 'motorway'
iraq_roads_motorway = VectorFileIO(uri='../tests/data/iraq_roads.geojson', 
                          filters=[('type','=','motorway')])

# Motorway A1
iraq_roads_a1 = VectorFileIO(uri='../tests/data/iraq_roads.geojson', 
                          filters=[('ref','=','A1')])

# Bike paths in Iraq, i.e. roads where type = 'bikepath'; 
# use only a subset of available columns
iraq_bikepaths = VectorFileIO(uri='../tests/data/iraq_roads.geojson', 
                              filters=[('type','=','bikepath')])

# Hospitals in Iraq
iraq_hospitals = VectorFileIO(uri='../tests/data/iraq_hospitals.geojson', columns=['name'])

# Districts in Baghdad
baghdad_alldistricts = VectorFileIO(uri='../tests/data/baghdad_districts.geojson')

# Districts in Baghdad where name starts with 'A'
baghdad_districts_a = VectorFileIO(uri='../tests/data/baghdad_districts.geojson',
                                   filters=[('NNAME', 'startswith', 'A')])

# A single point
featureio = FeatureIO(features=[
        {"geometry": {"type": "Point", "coordinates": [44.43, 33.33]}, "properties":{}},
    ])

# Raster images
globaltemp = RasterFileIO(uri='../tests/data/globalairtemp.tif')
globalprecip = RasterFileIO(uri='../tests/data/globalprecip.tif')

Verify that the 'iraq_bikepaths' IO object only contains records where type = 'bikepath'

In [3]:
iraq_bikepaths.read()
iraq_bikepaths.data

,bridge,geometry,maxspeed,name,oneway,osm_id,ref,tunnel,type
0,0,"(LINESTRING (44.3087191 33.2924177, 44.3081596...",0,,1,4061247,A86,0,bikepath


Verify that the 'baghdad_districts_a' IO object only contains records where nname starts with 'A'

In [4]:
baghdad_districts_a.read()
baghdad_districts_a.data

,DID,DNAME,NID,NNAME,geometry,id
5,3,7 Nissan,36,"Al Husseinia, Maamal, Ar Rashad",(POLYGON ((44.49221756429608 33.33403908470888...,baghdad_neighborhoods.6
21,1,Adhamiyah,14,Adhamiyah,(POLYGON ((44.37900477493537 33.35405573255764...,baghdad_neighborhoods.22
36,0,Rusafa,5,Al Saadoom Park,(POLYGON ((44.42613457929622 33.31509845782546...,baghdad_neighborhoods.37
40,0,Rusafa,3,Abu Nuwas,"(POLYGON ((44.423609555079 33.31177762337143, ...",baghdad_neighborhoods.41
45,6,Kadhimiyah,58,"Ali Al-Saleh, Salam",(POLYGON ((44.32730297460603 33.34064005941366...,baghdad_neighborhoods.66
55,5,Karkh,52,"Al-Kindi, Harithiya",(POLYGON ((44.39937146502945 33.30226725093627...,baghdad_neighborhoods.55
60,7,Mansour,69,Al Amriya,(POLYGON ((44.31444838250417 33.30895203324138...,baghdad_neighborhoods.60
67,7,Mansour,71,Al-Adil,"(POLYGON ((44.32471865292229 33.3297944594068,...",baghdad_neighborhoods.68
71,7,Mansour,67,Al Yarmuk,(POLYGON ((44.32310345187011 33.29070310243293...,baghdad_neighborhoods.72
73,7,Mansour,70,"Al Khadhra'a, Al Jamia'a",(POLYGON ((44.32940273597404 33.32739541078479...,baghdad_neighborhoods.74


## Area Process

Calculate the area of districts in Baghdad.  The output will be in square meters.  Display first 5 records.

In [5]:
area_pr = AreaProcess(inputs=[baghdad_alldistricts])
area_pr.compute()
print area_pr.output.uri
area_pr.output.data[:5]

/Users/mbertrand/dev/kitware/gaia/tests/data/output/ff5fd222-e009-4d0a-b433-913d694024c0/ff5fd222-e009-4d0a-b433-913d694024c0.json


,DID,DNAME,NID,NNAME,geometry,id,area
0,7,Mansour,74,Baghdad International Airport,(POLYGON ((44.25416164813192 33.36238209600769...,baghdad_neighborhoods.1,8.613726e+07
1,4,Karadah,46,Zafriniya,(POLYGON ((44.51697757715969 33.23343163393921...,baghdad_neighborhoods.2,1.296147e+07
2,1,Adhamiyah,25,Rashdiya,(POLYGON ((44.38008196547639 33.43048095703135...,baghdad_neighborhoods.3,1.038663e+07
3,2,Thawra,28,Thawra City,"(POLYGON ((44.460606439164 33.35860356319701, ...",baghdad_neighborhoods.4,3.312065e+07
4,3,7 Nissan,33,"Habibiyah, Dor Al-Umal, Baladiyat",(POLYGON ((44.46060643916433 33.35860356319682...,baghdad_neighborhoods.5,1.138407e+07


Load the above results from the saved output GeoJSON file and display the properties of the first record

In [6]:
with open(area_pr.output.uri, 'r') as infile:
    result_json = json.load(infile)
    for property in result_json['features'][0]['properties'].items():
        print '{}: {}'.format(property[0], property[1])

area: 86137256.6561
DID: 7
NID: 74
DNAME: Mansour
id: baghdad_neighborhoods.1
NNAME: Baghdad International Airport


## Length Process

Calculate the lengths of roads in Iraq, and display the first 5 records.

In [7]:
length_pr = LengthProcess(inputs=[iraq_allroads])
length_pr.compute()
print length_pr.output.uri
length_pr.output.data[:5]

/Users/mbertrand/dev/kitware/gaia/tests/data/output/851ff6a5-082d-44c7-9282-378b3bdf3903/851ff6a5-082d-44c7-9282-378b3bdf3903.json


,bridge,geometry,maxspeed,name,oneway,osm_id,ref,tunnel,type,length
0,0,"(LINESTRING (44.30871909999998 33.2924177, 44....",0,,1,4061247,A86,0,bikepath,4542.446040
1,0,"(LINESTRING (44.2976627 33.37263139999998, 44....",0,,1,4074629,A86,0,motorway,5745.322548
2,0,"(LINESTRING (44.39326450000001 33.3601687, 44....",0,,1,4074770,A86/N11/D383,0,motorway,1610.913903
3,0,"(LINESTRING (44.3933047 33.35987779999999, 44....",0,,1,4074779,A86/N11/D383,0,motorway,1092.254097
4,0,(LINESTRING (44.30015469999999 33.403856499999...,0,,1,4076793,A1,0,motorway,1587.980294


## BufferProcess

Buffer the bikepath in Iraq by 1000 meters

In [8]:
buffer_pr = BufferProcess(inputs=[iraq_bikepaths], buffer_size = 1000)
buffer_pr.compute()
print buffer_pr.output.uri

iraq_map = folium.Map([33.2924177, 44.30871909999998],
                  zoom_start=12,
                  tiles='cartodbpositron')

bikepath = folium.features.GeoJson(iraq_bikepaths.data.to_json())
buffer = folium.features.GeoJson(buffer_pr.output.data.to_json())

iraq_map.add_children(bikepath)
iraq_map.add_children(buffer)
iraq_map

/Users/mbertrand/dev/kitware/gaia/tests/data/output/89c8a915-12ef-42fe-bb83-a6a092791b79/89c8a915-12ef-42fe-bb83-a6a092791b79.json


## Within Process

Find hospitals in Iraq that are located inside Baghdad districts.

In [9]:
within_pr = WithinProcess(inputs=[iraq_hospitals, baghdad_alldistricts])
within_pr.compute()
print within_pr.output.uri
within_pr.output.data[:5]

/Users/mbertrand/dev/kitware/gaia/tests/data/output/d87bcf8d-454d-4273-88f5-c004b85aaded/d87bcf8d-454d-4273-88f5-c004b85aaded.json


,fid,geometry,id,name,osm_id,timestamp,type
0,28,POINT (44.3781274 33.3451792),iraq_hospitals.fid-76c404c7_15254deab67_-7748,جامعة بغداد -- كلية الطب آل,94298797,2012-07-20T11:15:39,hospital
9,165,POINT (44.3253978 33.3764162),iraq_hospitals.fid-76c404c7_15254deab67_-773f,,997416381,2010-11-21T05:22:05,hospital
10,166,POINT (44.3243249 33.377993),iraq_hospitals.fid-76c404c7_15254deab67_-773e,,997416382,2010-11-21T05:22:06,hospital
14,189,POINT (44.3562015 33.2550027),iraq_hospitals.fid-76c404c7_15254deab67_-773a,dental clinic dr.nadir,1037908152,2010-12-13T23:42:52,hospital
18,215,POINT (44.3602695 33.3171287),iraq_hospitals.fid-76c404c7_15254deab67_-7736,Iraqi Red Crescent Hospital,1167754844,2011-02-23T08:12:29,hospital


In [10]:
iraq_map = folium.Map([33.2924177, 44.30871909999998],
                  zoom_start=11,
                  tiles='cartodbpositron')

hospitals = folium.features.GeoJson(within_pr.output.data.to_json())
districts = folium.features.GeoJson(baghdad_alldistricts.data.to_json())

iraq_map.add_children(hospitals)
iraq_map.add_children(districts)
iraq_map

## Nearby Process

Find hospitals nearest to the bikepath, no more than 8km away.

In [11]:
nearby_pr = NearProcess(inputs=[iraq_hospitals, iraq_bikepaths], distance=8000)
nearby_pr.compute()
print nearby_pr.output.uri
nearby_pr.output.data

/Users/mbertrand/dev/kitware/gaia/tests/data/output/86313524-0636-4d7c-a49d-1e065dcf1bbd/86313524-0636-4d7c-a49d-1e065dcf1bbd.json


,fid,geometry,id,name,osm_id,timestamp,type,distance
18,215,POINT (44.36026949999999 33.31712869999998),iraq_hospitals.fid-76c404c7_15254deab67_-7736,Iraqi Red Crescent Hospital,1167754844,2011-02-23T08:12:29,hospital,6400.225937
14,189,POINT (44.3562015 33.25500269999997),iraq_hospitals.fid-76c404c7_15254deab67_-773a,dental clinic dr.nadir,1037908152,2010-12-13T23:42:52,hospital,7263.360569
9,165,POINT (44.32539779999999 33.37641619999999),iraq_hospitals.fid-76c404c7_15254deab67_-773f,,997416381,2010-11-21T05:22:05,hospital,7305.536711
10,166,POINT (44.32432489999999 33.37799299999999),iraq_hospitals.fid-76c404c7_15254deab67_-773e,,997416382,2010-11-21T05:22:06,hospital,7457.207820
29,396,POINT (44.32317389999999 33.23516890000001),iraq_hospitals.fid-76c404c7_15254deab67_-772b,Un Building,1966367994,2013-11-10T18:06:55,hospital,7789.696203
34,491,POINT (44.34124539999999 33.37471619999999),iraq_hospitals.fid-76c404c7_15254deab67_-7726,Al Kazimiyah,2202187519,2013-03-14T19:47:59,hospital,7952.363664


In [12]:
iraq_map = folium.Map([33.2924177, 44.30871909999998],
                  zoom_start=11,
                  tiles='cartodbpositron')

hospitals = folium.features.GeoJson(nearby_pr.output.data.to_json())
bikepath = folium.features.GeoJson(iraq_bikepaths.data.to_json())

iraq_map.add_children(hospitals)
iraq_map.add_children(bikepath)
iraq_map

## Distance Process

Calculate distance from each hospital to the nearest road.

In [13]:
distance_pr = DistanceProcess(inputs=[iraq_hospitals, iraq_allroads])
distance_pr.compute()
print distance_pr.output.uri
distance_pr.output.data[0:5]

/Users/mbertrand/dev/kitware/gaia/tests/data/output/577b3c74-0e5d-4e1c-ab17-bf9a746cee39/577b3c74-0e5d-4e1c-ab17-bf9a746cee39.json


,fid,geometry,id,name,osm_id,timestamp,type,distance
33,490,POINT (44.34352449999999 33.377348),iraq_hospitals.fid-76c404c7_15254deab67_-7727,Al Kazimiyah Pediatric,2202187518,2013-03-14T19:47:59,hospital,563.388416
10,166,POINT (44.32432489999999 33.37799299999999),iraq_hospitals.fid-76c404c7_15254deab67_-773e,,997416382,2010-11-21T05:22:06,hospital,855.158626
34,491,POINT (44.34124539999999 33.37471619999999),iraq_hospitals.fid-76c404c7_15254deab67_-7726,Al Kazimiyah,2202187519,2013-03-14T19:47:59,hospital,993.049032
9,165,POINT (44.32539779999999 33.37641619999999),iraq_hospitals.fid-76c404c7_15254deab67_-773f,,997416381,2010-11-21T05:22:05,hospital,1058.219341
35,492,POINT (44.3796559 33.36271089999999),iraq_hospitals.fid-76c404c7_15254deab67_-7725,,2202187574,2013-03-14T19:48:01,hospital,1115.128166


## Intersects Process

Find the districts that intersect the motorways.

In [14]:
intersects_pr = IntersectsProcess(inputs=[baghdad_alldistricts, iraq_roads_motorway])
intersects_pr.compute()
print intersects_pr.output.uri
intersects_pr.output.data[:5]

/Users/mbertrand/dev/kitware/gaia/tests/data/output/b384f794-e348-4944-b771-0a8cc4ea8e95/b384f794-e348-4944-b771-0a8cc4ea8e95.json


,DID,DNAME,NID,NNAME,geometry,id
17,0,Rusafa,7,Sheik Omar,(POLYGON ((44.38829821861083 33.34871127880871...,baghdad_neighborhoods.18
19,1,Adhamiyah,17,Maghreb,(POLYGON ((44.38642591694281 33.39479094110987...,baghdad_neighborhoods.20
20,1,Adhamiyah,20,Tunis,"(POLYGON ((44.36199641275005 33.3915827325705,...",baghdad_neighborhoods.21
25,0,Rusafa,10,Mustansiriya,"(POLYGON ((44.41320658451895 33.3491355477567,...",baghdad_neighborhoods.26
26,1,Adhamiyah,16,Waziriya-Industry,"(POLYGON ((44.39382687626517 33.3590579756513,...",baghdad_neighborhoods.27


In [15]:
iraq_map = folium.Map([33.35, 44.30871909999998],
                  zoom_start=12,
                  tiles='cartodbpositron')

linestyle = lambda x: {'color': 'red'}
districts = folium.features.GeoJson(intersects_pr.output.data.to_json())
motorways = folium.features.GeoJson(iraq_roads_motorway.data.to_json(), style_function=linestyle)

iraq_map.add_children(districts)
iraq_map.add_children(motorways)
iraq_map

## Disjoint

Find the districts that do not intersect the motorways.

In [16]:
disjoint_pr = DisjointProcess(inputs=[baghdad_alldistricts, iraq_roads_motorway])
disjoint_pr.compute()
disjoint_pr.output.data[:5]

,DID,DNAME,NID,NNAME,geometry,id
0,7,Mansour,74,Baghdad International Airport,(POLYGON ((44.25416164813191 33.36238209600771...,baghdad_neighborhoods.1
1,4,Karadah,46,Zafriniya,(POLYGON ((44.51697757715971 33.23343163393923...,baghdad_neighborhoods.2
2,1,Adhamiyah,25,Rashdiya,(POLYGON ((44.38008196547639 33.43048095703136...,baghdad_neighborhoods.3
3,2,Thawra,28,Thawra City,"(POLYGON ((44.460606439164 33.35860356319699, ...",baghdad_neighborhoods.4
4,3,7 Nissan,33,"Habibiyah, Dor Al-Umal, Baladiyat",(POLYGON ((44.46060643916434 33.35860356319682...,baghdad_neighborhoods.5


In [17]:
iraq_map = folium.Map([33.35, 44.30871909999998],
                  zoom_start=11,
                  tiles='cartodbpositron')

linestyle = lambda x: {'color': 'red'}
districts = folium.features.GeoJson(disjoint_pr.output.data.to_json())
motorways = folium.features.GeoJson(iraq_roads_motorway.data.to_json(), style_function=linestyle)

iraq_map.add_children(districts)
iraq_map.add_children(motorways)
iraq_map

## Crosses

Find the districts that cross the motorways.

Definition of 'cross': [the interior of a feature intersects the interior of another but does not contain it, and the dimension of the intersection is less than the dimension of the one or the other.](http://geopandas.org/reference.html#GeoSeries.crosses)

In [18]:
crosses_pr = CrossesProcess(inputs=[baghdad_alldistricts, iraq_roads_motorway])
crosses_pr.compute()
print crosses_pr.output.uri
crosses_pr.output.data[:5]

/Users/mbertrand/dev/kitware/gaia/tests/data/output/3deb43a0-cdaf-4e4f-9257-27f6b477a230/3deb43a0-cdaf-4e4f-9257-27f6b477a230.json


,DID,DNAME,NID,NNAME,geometry,id
17,0,Rusafa,7,Sheik Omar,(POLYGON ((44.38829821861083 33.34871127880871...,baghdad_neighborhoods.18
19,1,Adhamiyah,17,Maghreb,(POLYGON ((44.38642591694281 33.39479094110987...,baghdad_neighborhoods.20
20,1,Adhamiyah,20,Tunis,"(POLYGON ((44.36199641275005 33.3915827325705,...",baghdad_neighborhoods.21
25,0,Rusafa,10,Mustansiriya,"(POLYGON ((44.41320658451895 33.3491355477567,...",baghdad_neighborhoods.26
26,1,Adhamiyah,16,Waziriya-Industry,"(POLYGON ((44.39382687626517 33.3590579756513,...",baghdad_neighborhoods.27


In [19]:
iraq_map = folium.Map([33.35, 44.30871909999998],
                  zoom_start=12,
                  tiles='cartodbpositron')

linestyle = lambda x: {'color': 'red'}
districts = folium.features.GeoJson(crosses_pr.output.data.to_json())
motorways = folium.features.GeoJson(iraq_roads_motorway.data.to_json(), style_function=linestyle)

iraq_map.add_children(districts)
iraq_map.add_children(motorways)
iraq_map

## Touches

Find the motorways that touch edges with but do not overlap the bike path.

In [20]:
touches_pr = TouchesProcess(inputs=[iraq_roads_motorway, iraq_bikepaths])
touches_pr.compute()
print touches_pr.output.uri
touches_pr.output.data

/Users/mbertrand/dev/kitware/gaia/tests/data/output/a5fc4849-18e0-438f-bbc0-0144ebc396d9/a5fc4849-18e0-438f-bbc0-0144ebc396d9.json


,bridge,geometry,maxspeed,name,oneway,osm_id,ref,tunnel,type
37,1,"(LINESTRING (44.3005238 33.3256872, 44.3004093...",0,,1,224040659,A86,0,motorway


In [21]:
iraq_map = folium.Map([33.35, 44.30871909999998],
                  zoom_start=12,
                  tiles='cartodbpositron')

linestyle = lambda x: {'color': 'red'}
bikepaths = folium.features.GeoJson(iraq_bikepaths.data.to_json())
roads = folium.features.GeoJson(touches_pr.output.data.to_json(), style_function=linestyle)

iraq_map.add_children(bikepaths)
iraq_map.add_children(roads)
iraq_map

## Union

Create a dataset containing both the A1 motorway and the bike path.

In [41]:
union_pr = UnionProcess(inputs=[iraq_roads_a1, iraq_bikepaths])
union_pr.compute()
print union_pr.output.uri
union_pr.output.data

/Users/mbertrand/dev/kitware/gaia/tests/data/output/068836a9-087a-4da5-a8fc-49bd67379612/068836a9-087a-4da5-a8fc-49bd67379612.json


,bridge,geometry,maxspeed,name,oneway,osm_id,ref,tunnel,type
4,0,"(LINESTRING (44.3001547 33.4038565, 44.301367 ...",0,,1,4076793,A1,0,motorway
8,0,"(LINESTRING (44.3074825 33.3936365, 44.3066993...",0,,1,4076792,A1,0,motorway
14,0,"(LINESTRING (44.3172727 33.3805368, 44.3154935...",0,,1,4234529,A1,0,motorway
15,0,"(LINESTRING (44.3073645 33.3935827, 44.3115358...",0,,1,4234530,A1,0,motorway
41,0,"(LINESTRING (44.3145438 33.3837802, 44.3150089...",0,,1,233575082,A1,0,motorway
0,0,"(LINESTRING (44.3087191 33.2924177, 44.3081596...",0,,1,4061247,A86,0,bikepath


In [23]:
iraq_map = folium.Map([33.35, 44.30871909999998],
                  zoom_start=12,
                  tiles='cartodbpositron')

roadstyle = lambda x: {'color': 'red', 'line_opacity': 0.1, 'line_weight':15}
bikestyle = lambda y: {'color': 'green', 'line_opacity': 0.1, 'line_weight':10}
unionstyle = lambda z: {'color': 'blue', 'line_opacity': 0.1, 'line_weight':5}

bikepaths = folium.features.GeoJson(iraq_bikepaths.data.to_json(), style_function=bikestyle, name='Bikepaths')
roads = folium.features.GeoJson(iraq_roads_a1.data.to_json(), style_function=roadstyle, name='Motorways')
union = folium.features.GeoJson(union_pr.output.data.to_json(), style_function=unionstyle, name='Union')

iraq_map.add_children(roads)
iraq_map.add_children(bikepaths)
iraq_map.add_children(union)
folium.LayerControl().add_to(iraq_map)
iraq_map

## Centroid

Find the centroid of each district in Baghdad.

In [24]:
centroid_pr = CentroidProcess(inputs=[baghdad_alldistricts])
centroid_pr.compute()
print centroid_pr.output.uri
centroid_pr.output.data[0:5]

/Users/mbertrand/dev/kitware/gaia/tests/data/output/852668ae-9511-4e10-8ebc-914342cd5ab5/852668ae-9511-4e10-8ebc-914342cd5ab5.json


,DID,DNAME,NID,NNAME,geometry,id
0,7,Mansour,74,Baghdad International Airport,POINT (44.24203388449014 33.29812003683929),baghdad_neighborhoods.1
1,4,Karadah,46,Zafriniya,POINT (44.50798725653361 33.2531754261915),baghdad_neighborhoods.2
2,1,Adhamiyah,25,Rashdiya,POINT (44.36887089157968 33.41849950580148),baghdad_neighborhoods.3
3,2,Thawra,28,Thawra City,POINT (44.45881769675281 33.38902692655981),baghdad_neighborhoods.4
4,3,7 Nissan,33,"Habibiyah, Dor Al-Umal, Baladiyat",POINT (44.46994088448522 33.34038133876043),baghdad_neighborhoods.5


In [25]:
iraq_map = folium.Map([33.35, 44.30871909999998],
                  zoom_start=11,
                  tiles='cartodbpositron')

linestyle = lambda x: {'color': 'red'}
districts = folium.features.GeoJson(baghdad_alldistricts.data.to_json())
centroids = folium.features.GeoJson(centroid_pr.output.data.to_json())

iraq_map.add_children(districts)
iraq_map.add_children(centroids)
iraq_map

## Equals

Find the features in the allroads input that are the same as the features in the motorways input

In [26]:
equals_pr = EqualsProcess(inputs=[iraq_allroads, iraq_roads_a1])
equals_pr.compute()
print equals_pr.output.uri
equals_pr.output.data[0:5]

/Users/mbertrand/dev/kitware/gaia/tests/data/output/3c1ea7b5-5d6b-4635-b44a-664af73e55ed/3c1ea7b5-5d6b-4635-b44a-664af73e55ed.json


,bridge,geometry,maxspeed,name,oneway,osm_id,ref,tunnel,type
4,0,"(LINESTRING (44.3001547 33.4038565, 44.301367 ...",0,,1,4076793,A1,0,motorway
8,0,"(LINESTRING (44.3074825 33.3936365, 44.3066993...",0,,1,4076792,A1,0,motorway
14,0,"(LINESTRING (44.3172727 33.3805368, 44.3154935...",0,,1,4234529,A1,0,motorway
15,0,"(LINESTRING (44.3073645 33.3935827, 44.3115358...",0,,1,4234530,A1,0,motorway
41,0,"(LINESTRING (44.3145438 33.3837802, 44.3150089...",0,,1,233575082,A1,0,motorway


In [27]:
iraq_map = folium.Map([33.35, 44.30871909999998],
                  zoom_start=12,
                  tiles='cartodbpositron')

roadstyle = lambda x: {'color': 'red', 'line_opacity': 0.1, 'line_weight':15}
motorwaystyle = lambda y: {'color': 'green', 'line_opacity': 0.1, 'line_weight':10}
equalstyle = lambda z: {'color': 'blue', 'line_opacity': 0.1, 'line_weight':5}

allroads = folium.features.GeoJson(iraq_allroads.data.to_json(), style_function=roadstyle, name='All Roads')
motorways = folium.features.GeoJson(iraq_roads_a1.data.to_json(), style_function=motorwaystyle, name='Motorways')
equal = folium.features.GeoJson(equals_pr.output.data.to_json(), style_function=equalstyle, name='Equal')

iraq_map.add_children(allroads)
iraq_map.add_children(motorways)
iraq_map.add_children(equal)
folium.LayerControl().add_to(iraq_map)
iraq_map

## Zonal Statistics

Calculate global temperature statistics (mean, min, max, etc) within two vector polygons

In [28]:
# Two polygons
polygonio = FeatureIO(features=[
        {"geometry": {"type": "Polygon", "coordinates": [
                    [[-120,20], [-120,40], [-70,40], [-70,20], [-120,20]]
                ]}, "properties":{"id": "North polygon"}},
        {"geometry": {"type": "Polygon", "coordinates": [
                    [[120,-20], [120,-40], [70,-40], [70,-20], [120,-20]]
                ]}, "properties":{"id": "South polygon"}}        
    ])
zonal_pr = ZonalStatsProcess(inputs=[globaltemp, polygonio])
zonal_pr.compute()
print zonal_pr.output.uri

/Users/mbertrand/dev/kitware/gaia/tests/data/output/69e55fc8-8a62-4b5c-9341-a23ad1df2bf2/69e55fc8-8a62-4b5c-9341-a23ad1df2bf2.json


In [29]:
zonal_pr.output.data

,count,geometry,id,max,mean,median,min,stddev,sum
0,1000,"POLYGON ((-120 20, -120 40, -70 40, -70 20, -1...",North polygon,29879.0,28286.520,28374.0,25939.0,941.351625,28286520.0
1,1000,"POLYGON ((120 -20, 120 -40, 70 -40, 70 -20, 12...",South polygon,30986.0,29433.919,29426.0,28627.0,438.157292,29433919.0


In [35]:
world_map = folium.Map([0,0],
                  zoom_start=2,
                  tiles='cartodbpositron')

polys = folium.features.GeoJson(polygonio.data.to_json(), name='Polygon')
temparray = np.array(globaltemp.read().GetRasterBand(1).ReadAsArray())
overlay = ImageOverlay(temparray, [[-90.0,-180.0],[90.0,180.0]], mercator_project=True, control=True)
overlay.layer_name = 'Air Temp'

world_map.add_children(overlay)
world_map.add_children(polys)

folium.LayerControl().add_to(world_map)
world_map

## Subset Raster

Create a new global temperature image only containing the area inside a polygon.

In [36]:
# One polygon
polygonio1 = FeatureIO(features=[
        {"geometry": {"type": "Polygon", "coordinates": [
                    [[-120,20], [-120,40], [-70,40], [-70,20], [-120,20]]
                ]}, "properties":{"id": "North polygon"}}       
    ])
subset_pr = SubsetProcess(inputs=[globaltemp, polygonio1])
subset_pr.compute()

In [37]:
world_map = folium.Map([0,0],
                  zoom_start=2,
                  tiles='cartodbpositron')

polys = folium.features.GeoJson(polygonio1.data.to_json(), name='Polygon')
world_map.add_children(polys, name="Polygons")

subset_array = np.array(subset_pr.output.read().GetRasterBand(1).ReadAsArray())
overlay = ImageOverlay(subset_array, [[20.0,-120.0],[40.0,-70.0]], mercator_project=True, control=True)
overlay.layer_name = 'Air Temp Subset'

world_map.add_children(overlay)
folium.LayerControl().add_to(world_map)

world_map

## Raster Math

Add global temperature and precipitation image data together then divide by 2.

In [38]:
math_pr = RasterMathProcess(inputs=[globaltemp, globalprecip], calc='(A+B)/2')
math_pr.compute()

/Users/mbertrand/dev/kitware/gaia/gaia/geo/gdal_functions.py:405: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  nodatavalues == 1, band_vals == nodata_vals[i])


In [39]:
world_map = folium.Map([0,0],
                  zoom_start=2,
                  tiles='cartodbpositron')

temparray = np.array(globaltemp.read().GetRasterBand(1).ReadAsArray())
preciparray = np.array(globalprecip.read().GetRasterBand(1).ReadAsArray())
matharray = np.array(math_pr.output.read().GetRasterBand(1).ReadAsArray())

bounds = [[-90,-180],[90,180]]

overlay_temp = ImageOverlay(temparray, bounds, mercator_project=True, control=True)
overlay_temp.layer_name = 'Air Temperature'
overlay_precip = ImageOverlay(preciparray, bounds, mercator_project=True, control=True)
overlay_precip.layer_name = 'Precipitation'
overlay_math = ImageOverlay(matharray, bounds, mercator_project=True, control=True)
overlay_math.layer_name = '(Temp * Precipitation)/2'    
    
for array in (overlay_temp, overlay_precip, overlay_math):
    world_map.add_children(array)
world_map